# The Survival of Titanic Passengers

Hello! Here we aimed to predict whether the titanic passenger survive or not by looking on the passenger information. The first time that comes into my mind is that 'How cruel!' We have to predict which one survived which one dead. But again, this is for educational purposes, right? So, this information could be useful for improving how we dead with the data so that we could get a higher accuracy, OR by observing the relationship between the features, we might can prevent any future accident. At least we could be aware of that :D
<br>
<br>
The link to the dataset can be found on the following link : https://www.kaggle.com/c/titanic
<br>
Data Dictionary <br>
Variable	Definition	Key <br>
survival	Survival	0 = No, 1 = Yes
pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
sex	Sex	
Age	Age in years	
sibsp	# of siblings / spouses aboard the Titanic	
parch	# of parents / children aboard the Titanic	
ticket	Ticket number	
fare	Passenger fare	
cabin	Cabin number	
embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

In [96]:
############ Compulsory Standard Library #################
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
import math
import pickle

############ Sklearn pre-processing Library #################
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

############ Sklearn model Library #################
from sklearn.linear_model import LogisticRegression
# import sklearn.linear_model

## Load the Dataset

In [3]:
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")

## Data Description



In [4]:
print('# of Features : ', train_df.shape[1])
print('Train Data Samples : ', train_df.shape[0])
print('Test Data Samples : ', test_df.shape[0])

# of Features :  12
Train Data Samples :  891
Test Data Samples :  418


PassengerID : Unique Value
Name : Does not contribute to the result


## Preprocessing the Data

This function is for eliminating the 'NaN' value on the data

In [39]:
def nan_eliminate(X_data):
    data_mean = np.nanmean(X_data, axis=0)
#     for j in range(0,X_data.shape[1]):
    for i in range(0,X_data.shape[0]):
        if math.isnan(X_data[i]):
            X_data[i] = data_mean
    return X_data

In [22]:
#### 1. One hot encoding
#### 2. delete the matrix
#### 3. return the X_train matrix

def one_hot_encoding(X_data, dummy=True):
    one_hot_encoding_data = pd.get_dummies(X_data, dummy_na=dummy)
    return one_hot_encoding_data

In [7]:
# X_train = train_df.iloc[:,2:]
y_train = np.array(train_df.iloc[:, 1])
y_test = np.array(test_df.iloc[:,1])

print('Total positive sample : ', np.sum(y_train == 1))
print('Total negative sample : ', np.sum(y_train == 0))
print('The train data is unbalanced')

Total positive sample :  342
Total negative sample :  549
The train data is unbalanced


In [18]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [37]:
pclass = one_hot_encoding(train_df.iloc[:,2], dummy=False)
sex = one_hot_encoding(train_df.iloc[:,4], dummy=False)
sibling = one_hot_encoding(train_df.iloc[:,6], dummy=False)
parent = one_hot_encoding(train_df.iloc[:,7], dummy=False)
cabin = one_hot_encoding(train_df.iloc[:,10])
embarked = one_hot_encoding(train_df.iloc[:,11], dummy=False)

In [82]:
age_original = train_df.iloc[:,5]
fare_original = train_df.iloc[:,9]
age_previous = nan_eliminate(age_original)
fare_previous = nan_eliminate(fare_original)
age = []
fare = []


for i in age_previous:
    if i <= 10:
        group = 1
    elif i <= 20:
        group = 2
    elif i <= 30:
        group = 3
    elif i <= 40:
        group = 4
    elif i <= 50:
        group = 5
    elif i <= 60:
        group = 6
    elif i <= 70:
        group = 7
    else:
        group = 8
    age.append(group)

age = one_hot_encoding(age, dummy=False)
for i in fare_previous:
    if i <= 100:
        fare_group = 1
    else:
        fare_group = 2
        
    fare.append(fare_group)
fare = one_hot_encoding(fare, dummy=False)

In [92]:
X_train = np.concatenate((pclass, sex, age, sibling, parent, fare,embarked), axis=1)
# X_train = np.concatenate((pclass,), axis=1)
# X_train = np.concatenate((X_train, age), axis=1)
print(X_train.shape)

(891, 32)


In [94]:
print(pclass.shape)
print(sex.shape)
print(age.shape)
print(sibling.shape)
print(parent.shape)
print(fare.shape)
# print(cabin.shape)
print(embarked.shape)

(891, 3)
(891, 2)
(891, 8)
(891, 7)
(891, 7)
(891, 2)
(891, 3)


In [95]:
train_df["Name"]

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
5                                       Moran, Mr. James
6                                McCarthy, Mr. Timothy J
7                         Palsson, Master. Gosta Leonard
8      Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
9                    Nasser, Mrs. Nicholas (Adele Achem)
10                       Sandstrom, Miss. Marguerite Rut
11                              Bonnell, Miss. Elizabeth
12                        Saundercock, Mr. William Henry
13                           Andersson, Mr. Anders Johan
14                  Vestrom, Miss. Hulda Amanda Adolfina
15                      Hewlett, Mrs. (Mary D Kingcome) 
16                                  Rice, Master. Eugene
17                          Wil

In [171]:
token = r"(?u)\b[\w\'/]+\b"
cv = CountVectorizer(lowercase=True, binary=True, token_pattern=token)
name_token = cv.fit_transform(train_df["Name"])
name_list = cv.get_feature_names()

# for i in name_list:
#     print(i)

freq = np.sum(name_token, axis=0)
indices_descending = np.argsort(freq.A1)[::-1]
# for i in range(0, len(indices_descending)):
#     print(name_list[indices_descending[i]])
#     if "dr" in name_list[indices_descending[i]]:
#     print(name_list[indices_descending[i]],
#          '\t', freq[0, indices_descending[i]])

# for i in range(0, len(indices_descending)):
#     if "rose" in name_list[indices_descending[i]]:
#         print(indices_descending[i], name_list[indices_descending[i]])

# 1498
# 1499
william_alive = 0
william_dead = 0
for i in range(0, len(y_train)):
    if name_token[i, 1498] == 1 and y_train[i] == 1:
        william_alive = william_alive + 1
    elif name_token[i, 1498] == 1 and y_train[i] == 0:
        william_dead = william_dead + 1

print(william_alive, ' ', william_dead)

22   41


In [176]:
ticket_matrices = cv.fit_transform(train_df["Ticket"])
ticket_token = cv.get_feature_names()

[i for i in ticket_token]

['10482',
 '110152',
 '110413',
 '110465',
 '110564',
 '110813',
 '111240',
 '111320',
 '111361',
 '111369',
 '111426',
 '111427',
 '111428',
 '112050',
 '112052',
 '112053',
 '112058',
 '112059',
 '11206',
 '112277',
 '112379',
 '113028',
 '113043',
 '113050',
 '113051',
 '113055',
 '113056',
 '113059',
 '113501',
 '113503',
 '113505',
 '113509',
 '113510',
 '113514',
 '113572',
 '113760',
 '113767',
 '113773',
 '113776',
 '113781',
 '113783',
 '113784',
 '113786',
 '113787',
 '113788',
 '113789',
 '113792',
 '113794',
 '113796',
 '113798',
 '113800',
 '113803',
 '113804',
 '113806',
 '113807',
 '1166',
 '11668',
 '11751',
 '11752',
 '11753',
 '11755',
 '11765',
 '11767',
 '11769',
 '11771',
 '11774',
 '11813',
 '11967',
 '12233',
 '12460',
 '12749',
 '12750',
 '13032',
 '13049',
 '13213',
 '13214',
 '13502',
 '13507',
 '13509',
 '13528',
 '13529',
 '13531',
 '13567',
 '13568',
 '14208',
 '14258',
 '14263',
 '14311',
 '14312',
 '14313',
 '14879',
 '14885',
 '14973',
 '1585',
 '1601',
